In [1]:
%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

In [59]:
import getpass
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('final_proj-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '4g')
conf.set('spark.executor.instances', '15')
conf.set('spark.port.maxRetries', '100')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

<SparkContext master=yarn appName=final_proj-musuroi>

In [60]:
spark = SparkSession(sc)

In [61]:
df = spark.read.csv('/datasets/project/istdaten/*/*/*', sep=';', header=True)

In [62]:
columns = 'TripDate string, TripId string, OperatorId string, OperatorAbbrv string, OperatorName string, ProductId string, LineId string, LineType string, UmlaufId string, TransportType string, AdditionalTrip boolean, FailedTrip boolean, BPUIC string, StopName string, ArrivalTimeScheduled string, ArrivalTimeActual string, ArrivalTimeActualStatus string,     DepartureTimeScheduled string, DepartureTimeActual string, DepartureTimeActualStatus string, SkipStation boolean'
columns = list(map(lambda x: x.split()[0],columns.split(',')))

In [63]:
for old, new in zip(df.columns, columns):
    print(old, new)
    df = df.withColumnRenamed(old, new)

BETRIEBSTAG TripDate
FAHRT_BEZEICHNER TripId
BETREIBER_ID OperatorId
BETREIBER_ABK OperatorAbbrv
BETREIBER_NAME OperatorName
PRODUKT_ID ProductId
LINIEN_ID LineId
LINIEN_TEXT LineType
UMLAUF_ID UmlaufId
VERKEHRSMITTEL_TEXT TransportType
ZUSATZFAHRT_TF AdditionalTrip
FAELLT_AUS_TF FailedTrip
BPUIC BPUIC
HALTESTELLEN_NAME StopName
ANKUNFTSZEIT ArrivalTimeScheduled
AN_PROGNOSE ArrivalTimeActual
AN_PROGNOSE_STATUS ArrivalTimeActualStatus
ABFAHRTSZEIT DepartureTimeScheduled
AB_PROGNOSE DepartureTimeActual
AB_PROGNOSE_STATUS DepartureTimeActualStatus
DURCHFAHRT_TF SkipStation


In [65]:
df.head()

Row(TripDate='13.09.2017', TripId='80:06____:17010:000', OperatorId='80:06____', OperatorAbbrv='DB', OperatorName='DB Regio AG', ProductId='Zug', LineId='17010', LineType='RE', UmlaufId=None, TransportType='RE', AdditionalTrip='false', FailedTrip='false', BPUIC='8500090', StopName='Basel Bad Bf', ArrivalTimeScheduled=None, ArrivalTimeActual=None, ArrivalTimeActualStatus='PROGNOSE', DepartureTimeScheduled='13.09.2017 05:45', DepartureTimeActual='13.09.2017 05:45:00', DepartureTimeActualStatus='PROGNOSE', SkipStation='false')

In [66]:
df.count()

196232995

In [53]:
df.select("StopName").distinct().count()

10366